In [1]:
import matplotlib.pyplot as plt
import numpy as np
import importlib
import extract_gaze_stimulus
import sys 
sys.path.append("/Users/zacharykelly/Documents/MATLAB/projects/lightLogger/raspberry_pi_firmware/utility")
import Pi_util
import importlib
import scipy.io 
from scipy.signal import find_peaks
import matlab.engine
import tqdm 
import warnings
import virtual_foveation
import hdf5storage


Loading DLC 3.0.0rc8...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [ ]:
# Generate a playable video
path_to_recording_chunks: str = "/Volumes/T7 Shield/sam_gazecal_106"
output_dir: str = "./"
Pi_util.generate_playable_videos(path_to_recording_chunks, output_dir, apply_digital_gain=True, fill_missing_frames=True, debayer_images=True, pupil_image_rotation_correction=True)

In [ ]:
# Load in the frames of the recording 
world_frames: np.ndarray = Pi_util.destruct_video("./sam_gazecal_106/W.avi", is_grayscale=True)
print(world_frames.shape)

In [ ]:
# Allocate an array of transformed frames 
transformed_world_frames: np.ndarray = np.zeros_like(world_frames)

In [ ]:
# Load in the pupil gaze angles 
gaze_angles: np.ndarray = np.nan_to_num(scipy.io.loadmat("./gaze_angles")["gaze_angles"], 0)
print(gaze_angles.shape)


In [ ]:
# Load in the first chunk of both cameras. This will tell us if one or the other started first 
world_start_chunk_metadata: np.array = np.load("/Volumes/T7 Shield/sam_gazecal_106/world_time_2025DASH10DASH06_09COLON26COLON14DOT693525_chunk_0_metadata.npy")
pupil_start_chunk_metadata: np.array = np.load("/Volumes/T7 Shield/sam_gazecal_106/pupil_time_2025DASH10DASH06_09COLON26COLON15DOT252214_chunk_0_metadata.npy")

In [ ]:
# Find the missing number of frames between the two measurements in time 
FPS: float = 120
missing_frames: int = (abs(len(world_frames) - len(gaze_angles)))
missing_time: float = missing_frames / FPS 
print(missing_frames)
print(missing_time)

# Print the start time of both cameras so we can see whcih is first 
print(world_start_chunk_metadata[0, 0] / (10 ** 9))
print(pupil_start_chunk_metadata[0, 0])

# Find the difference explained by this 
start_time_delay_time: float = abs(pupil_start_chunk_metadata[0, 0] - world_start_chunk_metadata[0, 0] / (10 ** 9))
start_time_delay_frames: int = int(np.ceil(start_time_delay_time * FPS))
print(start_time_delay_time)
print(start_time_delay_frames)

# Account for the fact that we measured the pupil camera time wise is 0.005 phase advanced 
pupil_phase_offset_seconds: float = 0.005
start_time_delay_time -= pupil_phase_offset_seconds
start_time_delay_frames -= int(np.ceil(pupil_phase_offset_seconds * FPS))

In [ ]:
# Start a new MATLAB session
eng: object = matlab.engine.start_matlab()
eng.tbUseProject("lightLoggerAnalysis", nargout=0)


In [ ]:
# Initialize the objects we will use 
intrinsics_path: str = "/Users/zacharykelly/Documents/MATLAB/projects/lightLoggerAnalysis/code/virtual_foveation_wip/intrinsics_calibration.mat"
transformation_path: str = "/Users/zacharykelly/Documents/MATLAB/projects/lightLoggerAnalysis/code/virtual_foveation_wip/perspective_transform"

intrinsics: dict = scipy.io.loadmat(intrinsics_path)["camera_intrinsics_calibration"]
transformation: dict = scipy.io.loadmat(transformation_path)


In [ ]:
print(transformation["perspective_transform"])

In [ ]:
# Iterate over the world frames 
frames_for_video: int = len(world_frames) #(start_time_delay_frames) + (120 * 5) # len(world_frames)
for world_frame_num in tqdm.trange(start_time_delay_frames, frames_for_video):    
    print(f"Processing frame: {world_frame_num+1}/{frames_for_video}", flush=True)

    # Retrieve the pupil frame that corresponds to this world camera frame 
    pupil_frame_num: int = world_frame_num - start_time_delay_frames

    # If we have gone out of bounds for pupil, simply quit 
    if(pupil_frame_num >= len(gaze_angles)):
        warnings.warn(f"{pupil_frame_num} Out of bounds for pupil frame, breaking")
        break 

    # Otherwise, retrieve the gaze angles for this pupil frame 
    # and world frame 
    world_frame: np.ndarray = world_frames[world_frame_num]
    pupil_gaze_angles: np.ndarray = gaze_angles[pupil_frame_num, :2]
    
    # If there is nan gaze angles, just skip this frame 
    # and leave it all 0s
    if(np.any(np.isnan(pupil_gaze_angles))):
        continue

    # Then feed this as input into MATLAB to generate the correced frame 
    transformed_world_frame: np.ndarray = np.array(eng.coordinateTransformFinal(matlab.double(np.ascontiguousarray(world_frame).astype(np.float64)),
                                                                                intrinsics_path, 
                                                                                transformation_path,
                                                                                matlab.double(np.ascontiguousarray(pupil_gaze_angles[::-1].astype(np.float64))),
                                                                                nargout=1
                                                                               ))
    transformed_world_frame = np.clip(transformed_world_frame * 255, 0, 255).astype(np.uint8)    
    transformed_world_frame = np.flipud(transformed_world_frame)

    transformed_world_frames[world_frame_num] = transformed_world_frame


In [ ]:
# Close the MATLAB engine 
eng.close() 

In [ ]:

Pi_util.frames_to_video(transformed_world_frames, output_path="virtually_foveated_video.avi", fps=120)